## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
combined_df = pd.merge(mouse_metadata,study_results,on="Mouse ID")

# Display the data table for preview
combined_df.head()


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [2]:
# Checking the number of mice.
combined_df["Mouse ID"].value_counts().head()

g989    13
o795    10
c302    10
e584    10
q511    10
Name: Mouse ID, dtype: int64

In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duple_mouse = combined_df.set_index("Mouse ID")
mouse_time = duple_mouse.loc["g989", "Timepoint"]
print(mouse_time)


Mouse ID
g989     0
g989     0
g989     5
g989     5
g989    10
g989    10
g989    15
g989    15
g989    20
g989    20
g989    25
g989    30
g989    35
Name: Timepoint, dtype: int64


In [4]:
# Optional: Get all the data for the duplicate mouse ID. 
multi_mouse = combined_df.set_index("Mouse ID")
bad_mouse = multi_mouse.loc["g989", :]
print(bad_mouse)


         Drug Regimen     Sex  Age_months  Weight (g)  Timepoint  \
Mouse ID                                                           
g989         Propriva  Female          21          26          0   
g989         Propriva  Female          21          26          0   
g989         Propriva  Female          21          26          5   
g989         Propriva  Female          21          26          5   
g989         Propriva  Female          21          26         10   
g989         Propriva  Female          21          26         10   
g989         Propriva  Female          21          26         15   
g989         Propriva  Female          21          26         15   
g989         Propriva  Female          21          26         20   
g989         Propriva  Female          21          26         20   
g989         Propriva  Female          21          26         25   
g989         Propriva  Female          21          26         30   
g989         Propriva  Female          21       

In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
combined_df.sort_values("Mouse ID")
combined_df.drop_duplicates("Mouse ID", keep="first")
print(combined_df)

     Mouse ID Drug Regimen   Sex  Age_months  Weight (g)  Timepoint  \
0        k403     Ramicane  Male          21          16          0   
1        k403     Ramicane  Male          21          16          5   
2        k403     Ramicane  Male          21          16         10   
3        k403     Ramicane  Male          21          16         15   
4        k403     Ramicane  Male          21          16         20   
...       ...          ...   ...         ...         ...        ...   
1888     z969     Naftisol  Male           9          30         25   
1889     z969     Naftisol  Male           9          30         30   
1890     z969     Naftisol  Male           9          30         35   
1891     z969     Naftisol  Male           9          30         40   
1892     z969     Naftisol  Male           9          30         45   

      Tumor Volume (mm3)  Metastatic Sites  
0              45.000000                 0  
1              38.825898                 0  
2           

In [25]:
# Checking the number of mice in the clean DataFrame.
cleaned_df = combined_df.groupby(["Mouse ID"]).count()
cleaned_df.head()


,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
Mouse ID,,,,,,,
a203,10,10,10,10,10,10,10
a251,10,10,10,10,10,10,10
a262,10,10,10,10,10,10,10
a275,10,10,10,10,10,10,10
a366,7,7,7,7,7,7,7


## Summary Statistics

In [28]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

mean_tumor_df = combined_df.rename(columns={"Tumor Volume (mm3)":"Tumor Volume"})
mean_tumor_df = pd.DataFrame(combined_df.groupby(["Drug Regimen"]).mean())
mean_tumor_df = mean_tumor_df.reset_index()
mean_tumor_df.head()

#sum_stat_df = pd.DataFrame(combined_df.groupby(["Drug Regimen]))
#sum_stat_df.head()
                                                
# tumor_mean = combined_df["Tumor Volume"].mean() 
# tumor_median = combined_df["Tumor Volume"].median()
# tumor_var = combined_df["Tumor Volume"].var()
# tumor_std = combined_df["Tumor Volume"].std()
# tumor_sem = combined_df["Tumor Volume"].sem()

#sum_stat_df.head()






,Drug Regimen,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,Capomulin,13.456522,19.965217,21.565217,40.675741,0.713043
1,Ceftamin,13.247191,27.398876,19.747191,52.591172,1.179775
2,Infubinol,16.230337,27.196629,18.174157,52.884795,0.960674
3,Ketapril,15.659574,27.861702,19.707447,55.235638,1.297872
4,Naftisol,12.000000,27.166667,19.623656,54.331565,1.182796


In [29]:
med_tumor_df = pd.DataFrame(combined_df.groupby(["Drug Regimen"]).median())
med_tumor_df = med_tumor_df.reset_index()
med_tumor_df.head()

,Drug Regimen,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,Capomulin,16.5,20.5,20.0,41.557809,0.0
1,Ceftamin,12.0,28.0,20.0,51.776157,1.0
2,Infubinol,20.0,27.0,15.0,51.820584,1.0
3,Ketapril,18.0,28.0,20.0,53.698743,1.0
4,Naftisol,9.0,27.0,20.0,52.509285,1.0


In [30]:
var_tumor_df = pd.DataFrame(combined_df.groupby(["Drug Regimen"]).var())
var_tumor_df = var_tumor_df.reset_index()
var_tumor_df.head()

,Drug Regimen,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,Capomulin,59.620372,7.466034,206.928043,24.947764,0.720790
1,Ceftamin,65.147591,2.501016,204.031772,39.290177,1.402527
2,Infubinol,56.404272,4.769028,181.534470,43.128684,1.054942
3,Ketapril,36.236432,3.392536,196.839089,68.553577,1.942883
4,Naftisol,45.102703,2.247748,201.208951,66.173479,1.479919


In [31]:
std_tumor_df = pd.DataFrame(combined_df.groupby(["Drug Regimen"]).std())
std_tumor_df = std_tumor_df.reset_index()
std_tumor_df.head()

,Drug Regimen,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,Capomulin,7.721423,2.732404,14.384994,4.994774,0.848993
1,Ceftamin,8.071406,1.581460,14.283969,6.268188,1.184283
2,Infubinol,7.510278,2.183810,13.473473,6.567243,1.027104
3,Ketapril,6.019670,1.841884,14.029935,8.279709,1.393873
4,Naftisol,6.715855,1.499249,14.184814,8.134708,1.216519


In [32]:
sem_tumor_df = pd.DataFrame(combined_df.groupby(["Drug Regimen"]).sem())
sem_tumor_df = sem_tumor_df.reset_index()
sem_tumor_df.head()

,Drug Regimen,Age_months,Metastatic Sites,Mouse ID,Sex,Timepoint,Tumor Volume (mm3),Weight (g)
0,Capomulin,0.509136,0.055981,NaN,NaN,0.948518,0.329346,0.180169
1,Ceftamin,0.604977,0.088766,NaN,NaN,1.070629,0.469821,0.118535
2,Infubinol,0.562919,0.076985,NaN,NaN,1.009879,0.492236,0.163684
3,Ketapril,0.439030,0.101659,NaN,NaN,1.023238,0.603860,0.134333
4,Naftisol,0.492430,0.089200,NaN,NaN,1.040081,0.596466,0.109930


In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
